<a href="https://colab.research.google.com/github/BatoulAllahham/fine-tune-Flan-T5-with-LoRA/blob/main/Final_Project_last_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project
## Fine- tune Flan-T5  the base model using LoRA


### `Install Required Libraries`

In [ ]:
!pip install -q torch transformers datasets peft accelerate bitsandbytes

%pip install -U \
    datasets \
    transformers==4.41.0 \
    accelerate==0.28.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    peft==0.3.0 \
    fsspec==2025.3.0 --quiet
# accelerate: it moves models and data to the right devices, handles mixed precision, and makes scaling from one GPU to many almost painless

!pip install --upgrade datasets ### if dataset throughs any error


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.1 MB/s eta 0:00:00


### `Import Libraries`

In [ ]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

### `Cofiguration`

In [ ]:
MAX_SOURCE_LENGTH = 512
MAX_TARGET_LENGTH = 128
PER_DEVICE_TRAIN_BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH_SIZE = 4

dataset_name = 'knkarthick/samsum'
model_name='google/flan-t5-base'
output_dir='flan-t5-samsum-lora'


### `Load Dataset`




In [ ]:
torch.cuda.empty_cache()

dataset = load_dataset(dataset_name)
dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

### `Load Model and Tokenizer`

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(torch.cuda.is_available()) # if it can see CUDA on my machine
print(torch.cuda.get_device_name(0))
print(next(model.parameters()).dtype)
print(next(model.parameters()).device)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

True
Tesla T4
torch.bfloat16
cpu


### `Tokenize Dataset`

In [ ]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return example

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','dialogue', 'summary'])

print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)


Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Shapes of the datasets:
Training: (14731, 2)
Validation: (818, 2)
Test: (819, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 818
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 819
    })
})


### `Configure LoRA for PEFT Fine-tuning`

In [ ]:
lora_configuration = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    target_modules=["q", "v"]
)

model = get_peft_model(model, lora_configuration)
model.config.use_cache = False
model.print_trainable_parameters()


trainable params: 3538944 || all params: 251116800 || trainable%: 1.4092820552029972


### `Define Training Arguments`

In [ ]:
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=50,
    logging_steps=50,
    report_to="none",
    max_steps=1000
)




`### Trainer`

In [ ]:
trainer = Trainer(
    model=model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
50,9.880300
100,0.444900
150,0.208900
200,0.171800
250,0.145700
300,0.133600
350,0.126700
400,0.126000
450,0.119900
500,0.115500


TrainOutput(global_step=1000, training_loss=0.629287353515625, metrics={'train_runtime': 2257.1859, 'train_samples_per_second': 3.544, 'train_steps_per_second': 0.443, 'total_flos': 5565031907328000.0, 'train_loss': 0.629287353515625, 'epoch': 0.5428881650380022})

### `Evaluation`

In [ ]:
# Save adapter-only weights
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
print("Saved PEFT adapter to:", output_dir)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# Use full dataset (no debug slicing)
train_ds = dataset["train"]
val_ds = dataset["validation"]

# ROUGE evaluation
rouge = evaluate.load("rouge")

def generate_predictions(texts, batch_size=8):
    preds = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        encoded = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_SOURCE_LENGTH
        ).to(device)

        gen_ids = model.generate(
            **encoded,
            max_length=MAX_TARGET_LENGTH,
            num_beams=4
        )

        preds.extend(tokenizer.batch_decode(gen_ids, skip_special_tokens=True))

    return preds

# Get texts and references
val_texts = val_ds["dialogue"] if "dialogue" in val_ds.column_names else val_ds["text"]
val_refs  = val_ds["summary"]  if "summary" in val_ds.column_names else val_ds["highlights"]

# Predictions + ROUGE
preds = generate_predictions(val_texts, batch_size=PER_DEVICE_EVAL_BATCH_SIZE)
results = rouge.compute(predictions=preds, references=val_refs)

# Pretty print
print("ROUGE scores:")
for metric, score in results.items():
    try:
        print(f"- {metric}: {float(score) * 100:.2f}%")
    except:
        print(f"- {metric}: {score}")


Saved PEFT adapter to: flan-t5-samsum-lora
device: cuda


ROUGE scores:
- rouge1: 46.30%
- rouge2: 22.50%
- rougeL: 38.22%
- rougeLsum: 38.17%
